# Training a Linear Model (predict BMI off of Height/Weight, should be a perfect regression!)

In [1]:
import torch
import copy
import syft as sy
from torch import nn
from torch import optim
hook = sy.TorchHook(torch)

In [2]:
alex = sy.VirtualWorker(hook, id='alex')
toby = sy.VirtualWorker(hook, id='toby')
jake = sy.VirtualWorker(hook, id='jake')
matt = sy.VirtualWorker(hook, id='matt')
shreyas = sy.VirtualWorker(hook, id='shreyas')

# secure worker node built as well for secure regression building
secure_machine = sy.VirtualWorker(hook, id='secure_machine')

In [3]:
# Preprocessing of making tensors, sending, making a grid

# initiate tensors
alex_data = torch.tensor([[21, 1, 3, 165, 71, 23]]).tag("#test2")
toby_data = torch.tensor([[19, 0, 1, 175, 73, 23]]).tag("#test2")
jake_data = torch.tensor([23, 1, 4, 123, 69, 18]).tag("#test2")
matt_data = torch.tensor([28, 0, 7, 137, 82, 14]).tag("#test2")
shreyas_data = torch.tensor([16, 1, 2, 167, 76, 20]).tag("#test2")

# Send the tensor to the right worker node
alex_tensor = alex_data.send(alex)
toby_tensor = toby_data.send(toby)
jake_tensor = jake_data.send(jake)
matt_tensor = matt_data.send(matt)
shreyas_tensor = shreyas_data.send(shreyas)

workers = [alex, toby, jake, matt, shreyas]
grid = sy.PrivateGridNetwork(*workers)

In [4]:
alex_outcome_data = torch.tensor([[1]])
toby_outcome_data = torch.tensor([[0]])
jake_outcome_data = torch.tensor([[1]])
matt_outcome_data = torch.tensor([[1]])
shreyas_outcome_data = torch.tensor([[0]])

alex_outcome = alex_outcome_data.send(alex)
toby_outcome = toby_outcome_data.send(toby)
jake_outcome = jake_outcome_data.send(jake)
matt_outcome = matt_outcome_data.send(matt)
shreyas_outcome = shreyas_outcome_data.send(shreyas)

In [5]:
model = nn.Linear(6, 1)
abc = torch.randn(1,1)
abc

tensor([[-0.6821]])

In [6]:

# modified from OpenMinded Pysyft Tutorial Part 04 - Federated Learning via a Trusted Aggregator
def train_model(iterations, worker_iterations):
    
    for iterator in range(iterations):
        
        toby_model = model.copy().send(toby)
        alex_model = model.copy().send(alex)
        jake_model = model.copy().send(jake)
        matt_model = model.copy().send(matt)
        shreyas_model = model.copy().send(shreyas)
        
        # create optimizer functions
        toby_opt = optim.SGD(params=toby_model.parameters(), lr=0.05)
        alex_opt = optim.SGD(params=alex_model.parameters(), lr=0.05)
        jake_opt = optim.SGD(params=jake_model.parameters(), lr=0.05)
        matt_opt = optim.SGD(params=matt_model.parameters(), lr=0.05)
        shreyas_opt = optim.SGD(params=shreyas_model.parameters(), lr=0.05)
        
        for wi in range(worker_iterations):
            
            # Train Toby's Model
            toby_opt.zero_grad()
            toby_pred = toby_model(toby_tensor.float())
            toby_loss = ((toby_pred - toby_outcome)**2).sum()
            toby_loss.backward()
            
            toby_opt.step()
            toby_loss = toby_loss.get().data
            
            # Train Alex's Model
            alex_opt.zero_grad()
            alex_pred = alex_model(alex_tensor.float())
            alex_loss = ((alex_pred - alex_outcome)**2).sum()
            alex_loss.backward()
            
            alex_opt.step()
            alex_loss = alex_loss.get().data
            
            # Train Jake's Model
            jake_opt.zero_grad()
            jake_pred = jake_model(jake_tensor.float())
            jake_loss = ((jake_pred - jake_outcome)**2).sum()
            jake_loss.backward()
            
            jake_opt.step()
            jake_loss = jake_loss.get().data
            
            # Train Matt's Model
            matt_opt.zero_grad()
            matt_pred = matt_model(matt_tensor.float())
            matt_loss = ((matt_pred - matt_outcome)**2).sum()
            matt_loss.backward()
            
            matt_opt.step()
            matt_loss = matt_loss.get().data
            
            # Train Shreyas's Model
            shreyas_opt.zero_grad()
            shreyas_pred = shreyas_model(shreyas_tensor.float())
            shreyas_loss = ((shreyas_pred - shreyas_outcome)**2).sum()
            shreyas_loss.backward()
            
            shreyas_opt.step()
            shreyas_loss = shreyas_loss.get().data
            
        # move the model to secure worker
        toby_model.move(secure_machine)
        jake_model.move(secure_machine)
        alex_model.move(secure_machine)
        matt_model.move(secure_machine)
        shreyas_model.move(secure_machine)
                        
        # set the weight and bias divided by number of models used
        with torch.no_grad():
            model.weight.set_(((toby_model.weight.data + jake_model.weight.data + alex_model.weight.data + matt_model.weight.data + shreyas_model.weight.data) / 5).get())
            model.bias.set_(((toby_model.bias.data + jake_model.bias.data + alex_model.bias.data + matt_model.bias.data + shreyas_model.bias.data) / 5).get())
            
        # printing loss
        print ("toby loss: " + str(toby_loss))
        print ("jake loss: " + str(jake_loss))
        print ("alex loss: " + str(alex_loss))
        print ("matt loss: " + str(matt_loss))
        print ("shreyas loss: " + str(shreyas_loss))
        
train_model(8, 4)



toby loss: tensor(7.6816e+23)
jake loss: tensor(1.3077e+20)
alex loss: tensor(2.5488e+23)
matt loss: tensor(9.7157e+20)
shreyas loss: tensor(3.6741e+23)
toby loss: tensor(inf)
jake loss: tensor(inf)
alex loss: tensor(inf)
matt loss: tensor(inf)
shreyas loss: tensor(inf)
toby loss: tensor(inf)
jake loss: tensor(inf)
alex loss: tensor(inf)
matt loss: tensor(inf)
shreyas loss: tensor(inf)
toby loss: tensor(nan)
jake loss: tensor(nan)
alex loss: tensor(nan)
matt loss: tensor(nan)
shreyas loss: tensor(nan)
toby loss: tensor(nan)
jake loss: tensor(nan)
alex loss: tensor(nan)
matt loss: tensor(nan)
shreyas loss: tensor(nan)
toby loss: tensor(nan)
jake loss: tensor(nan)
alex loss: tensor(nan)
matt loss: tensor(nan)
shreyas loss: tensor(nan)
toby loss: tensor(nan)
jake loss: tensor(nan)
alex loss: tensor(nan)
matt loss: tensor(nan)
shreyas loss: tensor(nan)
toby loss: tensor(nan)
jake loss: tensor(nan)
alex loss: tensor(nan)
matt loss: tensor(nan)
shreyas loss: tensor(nan)


In [7]:

# modified from OpenMinded Pysyft Tutorial Part 04 - Federated Learning via a Trusted Aggregator
def train_model(iterations, worker_iterations):
    
    for iterator in range(iterations):
        
        toby_model = model.copy().send(toby)
        alex_model = model.copy().send(alex)
        
        # create optimizer functions
        toby_opt = optim.SGD(params=toby_model.parameters(), lr=0.15)
        alex_opt = optim.SGD(params=alex_model.parameters(), lr=0.15)
        
        for wi in range(worker_iterations):
            
            # Train Toby's Model
            toby_opt.zero_grad()
            toby_pred = toby_model(toby_tensor.float())
            toby_loss = ((toby_pred - toby_outcome)**2).sum()
            toby_loss.backward()
            
            toby_opt.step()
            toby_loss = toby_loss.get().data
            
            # Train Alex's Model
            alex_opt.zero_grad()
            alex_pred = alex_model(alex_tensor.float())
            alex_loss = ((alex_pred - alex_outcome)**2).sum()
            alex_loss.backward()
            
            alex_opt.step()
            alex_loss = alex_loss.get().data
            
            
        # move the model to secure worker
        toby_model.move(secure_machine)
        alex_model.move(secure_machine)
                        
        # set the weight and bias divided by number of models used
        with torch.no_grad():
            model.weight.set_(((toby_model.weight.data + alex_model.weight.data) / 2).get())
            model.bias.set_(((toby_model.bias.data + alex_model.bias.data) / 2).get())
            
        # printing loss
        print ("toby loss: " + str(toby_loss.data))
        print ("alex loss: " + str(alex_loss.data))
        
train_model(8, 4)

# this can see how well the model worked
predictions = model(alex_data.float())
loss = ((predictions - toby_outcome_data) ** 2).sum()
print(predictions)
print(toby_outcome)
print(loss.data)

toby loss: tensor(nan)
alex loss: tensor(nan)
toby loss: tensor(nan)
alex loss: tensor(nan)
toby loss: tensor(nan)
alex loss: tensor(nan)
toby loss: tensor(nan)
alex loss: tensor(nan)
toby loss: tensor(nan)
alex loss: tensor(nan)
toby loss: tensor(nan)
alex loss: tensor(nan)
toby loss: tensor(nan)
alex loss: tensor(nan)
toby loss: tensor(nan)
alex loss: tensor(nan)
tensor([[nan]], grad_fn=<AddmmBackward>)
(Wrapper)>[PointerTensor | me:26442319493 -> toby:77611184004]
tensor(nan)
